# 1.1 Experiment: market basket data

## A. Import modules
## B. Prepare functions
## C. Load & prepare data
## D. Load test data
## E. Experiment:
### 1) SVD
### 2) word2vec
### 3) Bitmap
### 4) Poincare
## F. Results


# A. Import modules 

In [2]:
# import libs
import numpy as np
import time
import gensim
from collections import Counter
from itertools import combinations
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from math import log
from gensim.models import word2vec
from annoy import AnnoyIndex
import random
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import tensorflow
from keras.utils import to_categorical
import numpy as np
import math

results = []

Using TensorFlow backend.


# B. Prepare functions

In [3]:
def generate_svd(transactions_dict, patterns_dict, n_dim = 300, negative = False):
	data_list = [v for _,v in transactions_dict.items()]

	unigrams_cnt = Counter()
	bigrams_cnt = Counter()
	for text in data_list:
		for x in text:
			unigrams_cnt[x] += 1
		for x, y in map(sorted, combinations(text, 2)):
			bigrams_cnt[(x, y)] += 1

	id2uni = {}
	uni2id = {}
	it = 0

	for uni,_ in unigrams_cnt.items():
		id2uni[it] = uni
		uni2id[uni] = it
		it +=1


	sum_uni = float(sum(unigrams_cnt.values()))
	sum_bi = float(sum(bigrams_cnt.values()))

	data, rows, cols = [], [], []
	for (x, y), n in bigrams_cnt.items():
		rows.append(uni2id[x])
		cols.append(uni2id[y])
		data.append(log((n / sum_bi) / (unigrams_cnt[x] / sum_uni) / (unigrams_cnt[y] / sum_uni)))
	PMI = csc_matrix((data, (cols, rows)), shape = (len(unigrams_cnt), len(unigrams_cnt)))
	U,_,_ = svds(PMI, k = n_dim)
	norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
	U /= np.maximum(norms, 1e-7)
    
    

	result_t_dict = {}
	result_p_dict = {}

	for key in transactions_dict.keys():
		for product in transactions_dict[key]:
			temp = [U[uni2id[product]] for product in transactions_dict[key]]
			result_t_dict[key] = power_means([x for x in temp])

	if not negative:        
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([U[uni2id[product]] for product in patterns_dict[key]])
	else:
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([U[uni2id[product]] for product in patterns_dict[key][:-1]])
			result_p_dict[key] = [result_p_dict[key][i] - np.dot(U, U[uni2id[patterns_dict[key][-1][i]]]) for i in range (n_dim)]
    
	return (result_t_dict,result_p_dict)

In [4]:
def generate_word2vec(transactions_dict, patterns_dict, n_dim = 300, n_workers = 10, n_epochs = 20, negative = False):
	data_list = [v for _,v in transactions_dict.items()]
	window_size = max([len(x) for x in data_list])

	model = word2vec.Word2Vec(data_list, size = n_dim, window = window_size, min_count = 1, workers = n_workers)

	model.train(data_list, total_examples = len(data_list), epochs = n_epochs)

    
	result_t_dict = {}
	result_p_dict = {}

	for key in transactions_dict.keys():
		result_t_dict[key] = power_means([model[product] for product in transactions_dict[key]])

	if not negative:        
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([model[product] for product in patterns_dict[key]])
	else:
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([model[product] for product in patterns_dict[key][:-1]])
			result_p_dict[key] = [result_p_dict[key][i] - patterns_dict[key][-1][i] for i in range (n_dim)]
            
	return (result_t_dict,result_p_dict)

In [5]:
def generate_answers(patterns_dict, transactions_dict):
    answers = {}
    l = len([x for x in patterns_dict.keys()])/2

    for i,key in enumerate(patterns_dict.keys()):
        
        if (i%l == 0):
            print(i/len([key for key in patterns_dict.keys()]),time.time()-a_time)
            
        answers[key] = [t_key for t_key in transactions_dict.keys() if set(patterns_dict[key]) <= set(transactions_dict[t_key])]    
    
    return answers
    

In [6]:
def power_means(list_of_vectors, p = 1):
	data = np.array(list_of_vectors)

	return np.power(np.power(data,p).mean(axis=0), 1/p)

In [7]:

def generate_patterns(input_dict):
    output_dict = {}
    i = 0
    for key in input_dict.keys():
        for k in range(4):
            temp = combinations(input_dict[key],k)
            for y in temp:
                output_dict['p_'+str(i)] = [l for l in y]
                if output_dict['p_'+str(i)] == []:
                    output_dict.pop('p_'+str(i))
                
                i+=1
        
    
    return output_dict
    

def generate_transactions(input_dict):
    output_dict = {}
    random.seed(14231)
    i = 0
    for key1 in input_dict.keys():
        for item in input_dict[key1]:
            try:
                new_key = 'test_'+str(i)
                result = []
                temp = []



                key2 = random.sample(input_dict.keys(),1)[0]
                key3 = random.sample(input_dict.keys(),1)[0]
                temp.append(random.choice(input_dict[key2]))
                temp.append(random.choice(input_dict[key3]))
                it1 = np.random.choice(input_dict[key1])
                it2 = np.random.choice(input_dict[key1])
                temp.append(it1)
                temp.append(it2)
                np.random.shuffle(temp)
                result = temp[:2]+[item]+temp[2:4]
                result = list(set(result))
                output_dict[new_key] = result

                if output_dict[new_key] == []:
                    output_dict.pop(new_key)
            except:
                pass
            
            i+=1
    
    return output_dict


In [8]:

def add_zeros(input_list,l = 8):
    return input_list + [0 for i in range(l-len(input_list))]

def gen_index(input_data_dict,d=2):
    count = Counter([x for sublist in input_data_dict.values() for x in sublist])

    depth = int(np.ceil(math.log(len(count.keys()), d)))
    
    
    base_length = d**depth
    
    onehot_encoder = MultiLabelBinarizer()
    
    
    result = onehot_encoder.fit_transform(input_data_dict.values())
    
    result = [[x[int(d*i):int(d*(i+1))] for i in range(int(int(base_length)/d))] for x in result]
        
    result = [[signature2int(add_zeros([int(z) for z in y],l=d)) for y in x] for x in result]
    
    if depth != 1:
        for l in range(len(result)):
            obj = result[l]
            for i in range(depth,1,-1):
                new_level = []
                old_level_length = d**(i-1)
                old_level = obj[:old_level_length]
                new_level_length = d**(i-2)
                new_level = [signature2int([int(bool(x)) for x in old_level[d*j:d*(j+1)]]) for j in range(int(old_level_length/d))]

                for k in range(new_level_length):
                    result[l].insert(k,new_level[k])
    
    
    index_dict = {}
    result_dict = {}
    
    for i,key in enumerate(input_data_dict.keys()):
        row_dict = {}
        k = 0 
        
        for j in range(len(result[i])):
            if result[i][j] != 0:
                row_dict[k] = j
                k+= 1
            
        index_dict[key] = row_dict
        result[i] = [y for y in result[i] if y != 0]
        result_dict[key] = result[i]
        
        
    return (result_dict, index_dict)    


def signature2int(signature):

     return int(''.join(map(str, signature)), 2)
    


def skip(i_t,i_p,i):
    skip = 0
    while i_t[i+skip] != i_p[i]:
        skip += 1
    return skip   


def search(pattern,p_index,transaction,t_index):
    if transaction[0] & pattern[0] == pattern[0]:
        for i in range(1,len(pattern)):
            p = skip(t_index,p_index,i)
            if transaction[i+p] & pattern[i] != pattern[i]:
                return False
    else:
        return False
    return True 

# C. Load data

In [9]:
a_time = time.time()
data_df = pd.read_csv('order_products__train.csv', nrows = 5000)

data_dict = {}

for index,row in data_df.iterrows():
    if row['order_id'] not in data_dict.keys():
        data_dict[row['order_id']] = [str(row['product_id'])]
    else:
        data_dict[row['order_id']].append(str(row['product_id']))


print("Processing took {} s".format(time.time()-a_time))

Processing took 0.5542752742767334 s


# D. Generate test data

In [10]:
a_time = time.time()

patterns_dict = generate_patterns(data_dict)

patterns_1 = {key:patterns_dict[key] for key in patterns_dict.keys() if len(patterns_dict[key]) == 1}
p_length = len([key for key in patterns_1.keys()])

patterns_2 = {}
i = 0
for key in patterns_dict.keys():
    if len(patterns_dict[key]) == 2:
        patterns_2[key] = patterns_dict[key]
        i += 1
    if i == p_length:
        break

patterns_3 = {}
i = 0
for key in patterns_dict.keys():
    if len(patterns_dict[key]) == 3:
        patterns_3[key] = patterns_dict[key]
        i += 1
    if i == p_length:
        break

transactions_dict = generate_transactions(data_dict)

answers_1 = generate_answers(patterns_1, transactions_dict)
answers_2 = generate_answers(patterns_2, transactions_dict)
answers_3 = generate_answers(patterns_3, transactions_dict)

print("Processing took {} s".format(time.time()-a_time))

0.0 1.437549352645874
0.5 11.069574117660522
0.0 20.613356113433838
0.5 30.507405996322632
0.0 40.415767431259155
0.5 50.451255559921265
Processing took 60.33112573623657 s


# E. Experiment

## 1) SVD

In [71]:
a_time = time.time()
svd_dim = 10


(t_svd, p_svd) = generate_svd(transactions_dict, patterns_dict, n_dim = svd_dim)

print("Processing took {} s".format(time.time()-a_time))

# Last run time:
# 3) nrows = 50000 - 900 s / cut off = 0.37

Processing took 8.2823166847229 s


In [72]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(svd_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],t_svd[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 10.745524883270264 s


In [73]:
a_time = time.time()

correct_answers = 0
answers_counter = 0 

for y in patterns_1.keys():

    x = annoy_index.get_nns_by_vector(p_svd[y],n = len(answers_1[y]))
    correct_answers += len([z for z in answers_1[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1[y])


results.append({'timestamp':time.time(),'algorithm':'SVD','dimensions':svd_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [74]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2.keys():

    x = annoy_index.get_nns_by_vector(p_svd[y],n = len(answers_2[y]))
    correct_answers += len([z for z in answers_2[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2[y])
    
results.append({'timestamp':time.time(),'algorithm':'SVD','dimensions':svd_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

In [75]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_3.keys():

    x = annoy_index.get_nns_by_vector(p_svd[y],n = len(answers_3[y]))
    correct_answers += len([z for z in answers_3[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_3[y])

results.append({'timestamp':time.time(),'algorithm':'SVD','dimensions':svd_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':3})

## 2) word2vec

In [76]:
a_time = time.time()

word2vec_dim = 10

(t_word2vec, p_word2vec) = generate_word2vec(transactions_dict, patterns_dict, n_dim = word2vec_dim)

print("Processing took {} s".format(time.time()-a_time))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Processing took 12.4683198928833 s


In [77]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(word2vec_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],t_word2vec[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 10.859231233596802 s


In [78]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_1.keys():

    x = annoy_index.get_nns_by_vector(p_word2vec[y],n = len(answers_1[y]))
    correct_answers += len([z for z in answers_1[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1[y])

results.append({'timestamp':time.time(),'algorithm':'word2vec','dimensions':word2vec_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [79]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2.keys():

    x = annoy_index.get_nns_by_vector(p_word2vec[y],n = len(answers_2[y]))
    correct_answers += len([z for z in answers_2[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2[y])

results.append({'timestamp':time.time(),'algorithm':'word2vec','dimensions':word2vec_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

In [80]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_3.keys():

    x = annoy_index.get_nns_by_vector(p_word2vec[y],n = len(answers_3[y]))
    correct_answers += len([z for z in answers_3[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_3[y])

results.append({'timestamp':time.time(),'algorithm':'word2vec','dimensions':word2vec_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':3})

## 3) Bitmap index

In [11]:
a_time = time.time()

temp_dict = patterns_dict
temp_dict.update(transactions_dict)

t_index, i_index = gen_index(temp_dict, d = 64)

print("Index generated")

print("Processing took {} s".format(time.time()-a_time))

Index generated
Processing took 593.0563380718231 s


In [12]:
a_time = time.time()


test_dict = {}
test_i = {}
pattern_dict = {}
pattern_i = {}

# test_keys = [key for key in data_dict.keys() if str(key)[0] == 't']

for key in t_index.keys():
    if str(key)[0] == 't':
        test_dict[key] = t_index[key]
        test_i[key] = i_index[key]
    elif str(key)[0] == 'p':
        pattern_dict[key] = t_index[key]
        pattern_i[key] = i_index[key]
    

print("Processing took {} s".format(time.time() - a_time))

Processing took 0.57666015625 s


In [13]:
a_time = time.time()

for key in patterns_1.keys():
    k = len(answers_1[key])
    i = 0
    for t_key in test_dict.keys():
        if search(pattern_dict[key],pattern_i[key],test_dict[t_key],test_i[t_key]):
            i += 1            
            if i == k:
                break
        
results.append({'timestamp':time.time(),'algorithm':'bitmap','dimensions':'X','time':time.time()-a_time,'quality':1, 'n_patterns':p_length,'patterns_length':1})

In [14]:
a_time = time.time()

for key in patterns_2.keys():
    k = len(answers_2[key])
    i = 0
    for t_key in test_dict.keys():
        if search(pattern_dict[key],pattern_i[key],test_dict[t_key],test_i[t_key]):
            i += 1            
            if i == k:
                break
        
results.append({'timestamp':time.time(),'algorithm':'bitmap','dimensions':'X','time':time.time()-a_time,'quality':1, 'n_patterns':p_length,'patterns_length':2})

In [15]:
a_time = time.time()

for key in patterns_3.keys():
    k = len(answers_3[key])
    i = 0
    for t_key in test_dict.keys():
        if search(pattern_dict[key],pattern_i[key],test_dict[t_key],test_i[t_key]):
            i += 1            
            if i == k:
                break
        
results.append({'timestamp':time.time(),'algorithm':'bitmap','dimensions':'X','time':time.time()-a_time,'quality':1, 'n_patterns':p_length,'patterns_length':3})

## 4) Poincare

In [16]:
a_time = time.time()

poincare_embeddings = pd.read_csv("market_data_poincare_A_10k_100d.tsv", sep="\t",header=None)
poincare_dim = 100

poincare_rels_dict = {}

for _,row in poincare_embeddings.iterrows():
    poincare_rels_dict[str(row[0])[:-2]] = [row[x] for x in range(1,poincare_dim+1)]

    
patterns_poincare = {}
for key in patterns_dict.keys():
    if key != '':
        patterns_poincare[key] = power_means([poincare_rels_dict[obj] for obj in patterns_dict[key] if obj != ''])
    
    if patterns_poincare[key] == []:
        patterns_poincare.pop(key)
    if str(patterns_poincare[key]) == 'nan':
        patterns_poincare.pop(key)


transactions_poincare = {}
for key in transactions_dict.keys():
    if key != '':
        transactions_poincare[key] = power_means([poincare_rels_dict[obj] for obj in transactions_dict[key] if obj != ''])
    
    if transactions_poincare[key] == []:
        transactions_poincare.pop(key)
    if str(transactions_poincare[key]) == 'nan':
        transactions_poincare.pop(key)
        
print("Processing took {} s".format(time.time()-a_time))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Processing took 365.58866477012634 s


In [17]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(poincare_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],transactions_poincare[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 5.49362587928772 s


In [18]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_1.keys():

    x = annoy_index.get_nns_by_vector(patterns_poincare[y],n = len(answers_1[y]))
    correct_answers += len([z for z in answers_1[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1[y])

results.append({'timestamp':time.time(),'algorithm':'poincare','dimensions':poincare_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [19]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2.keys():

    x = annoy_index.get_nns_by_vector(patterns_poincare[y],n = len(answers_2[y]))
    correct_answers += len([z for z in answers_2[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2[y])


results.append({'timestamp':time.time(),'algorithm':'poincare','dimensions':poincare_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

In [20]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_3.keys():

    x = annoy_index.get_nns_by_vector(patterns_poincare[y],n = len(answers_3[y]))
    correct_answers += len([z for z in answers_3[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_3[y])

results.append({'timestamp':time.time(),'algorithm':'poincare','dimensions':poincare_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':3})

# E. Results

In [423]:
import json 

for line in results:
    if (json.loads(json.dumps(line))['algorithm']) == 'bitmap':
        print(line)

{'timestamp': 1570562800.1420643, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.6713907718658447, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 1}
{'timestamp': 1570562800.7061377, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.5577149391174316, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 2}
{'timestamp': 1570562801.3265765, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.6139039993286133, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 3}


In [86]:
results_df = pd.DataFrame(results)
results_df['mat'] = results_df.apply(lambda row: row['time']/row['n_patterns'], axis = 1)
results_df[results_df['n_patterns'] == 5000].sort_values(by=['algorithm','dimensions'])

,algorithm,dimensions,n_patterns,patterns_length,quality,time,timestamp,mat
36,SVD,10,5000,1,0.238002,27.850685,1.570989e+09,0.005570
37,SVD,10,5000,2,0.134873,1.613965,1.570989e+09,0.000323
38,SVD,10,5000,3,0.354839,0.170194,1.570989e+09,0.000034
6,SVD,20,5000,1,0.491452,22.986855,1.570989e+09,0.004597
7,SVD,20,5000,2,0.473441,1.255250,1.570989e+09,0.000251
8,SVD,20,5000,3,0.838710,0.164959,1.570989e+09,0.000033
12,SVD,50,5000,1,0.798069,18.043349,1.570989e+09,0.003609
13,SVD,50,5000,2,0.802771,0.938538,1.570989e+09,0.000188
14,SVD,50,5000,3,0.935484,0.173873,1.570989e+09,0.000035
18,SVD,100,5000,1,0.912075,15.520551,1.570989e+09,0.003104


In [87]:
results_df.to_csv('experiment1_results.csv',index=None)